In [1]:
#-------------------------------------------------------------------------
# Original:CS-340 Dashboard: Grazioso Salvare Animal Shelter
# CS-499 Refactor: Data Structures & Algorithms
# Author: Andrew Allen
# Date: 2/8/2026
# Description: An interactive Dashboard which incorporates the AAC (Austin Animal Center) database via MongoDB. 
# 
#-------------------------------------------------------------------------
#imports needed to compile
import dash
import dash_leaflet as dl
from dash import dcc, html, dash_table, Input, Output, State
import plotly.express as px
import base64
import pandas as pd
import numpy as np
import os
#https://docs.scipy.org/doc/scipy/reference/spatial.html
from scipy.spatial import KDTree
#needed for new enhancement to original dashboard
from dash.exceptions import PreventUpdate

# Imported basic Crud Module
# Ensure CRUD_Python_Module.py is in the same directory
try:
    from CRUD_Python_Module import AnimalShelter
except ImportError:
    print("Error: CRUD_Python_Module.py not found. Please ensure the module is in the local directory.")

###########################
# Data Manipulation / Model
###########################

# CS-340: Database Connection Parameters
username = "aacuser"
password = "SNHU1234"
host = 'localhost'
database = 'AAC'
port = 27017
collection = 'animals'

# Initialize the CRUD connection
db = AnimalShelter(username, password, host, port, database, collection)

# Initial Data Load and Pre-processing
# drop the MongoDB ObjectID immediately to prevent Dash Table serialization errors
df = pd.DataFrame.from_records(db.read({}))

if not df.empty:
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    
    # Pre-processing coordinates for the K-D Tree spatial search algorithm
    # We force numeric types to ensure the spatial search doesn't crash on dirty data
    df['location_lat'] = pd.to_numeric(df['location_lat'], errors='coerce')
    df['location_long'] = pd.to_numeric(df['location_long'], errors='coerce')
    df = df.dropna(subset=['location_lat', 'location_long'])

##################################
# Dashboard Layout / View HTML/CSS
#################################
app = dash.Dash(__name__)

#try block in case png picture does not load properly for the assignment
try:
    image_filename = 'Grazioso Salvare Logo.png' 
    encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()
except FileNotFoundError:
    print(f"Warning: {image_filename} not found.")
    encoded_image = ""
#Root layer container, minheight/width is just for background color
app.layout = html.Div(style={'minHeight': '100vh', 'minWidth': '1280px', 'backgroundColor': '#ffe4e1', 'padding': '20px', 'fontFamily': 'helvetica'}, children=[
    
    # HEADER SECTION
    #contains the png as well as my name 
    html.Div(style={'display': 'flex', 'alignItems': 'center', 'marginBottom': '20px'}, children=[
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height':'80px'}),
        html.Div(style={'marginLeft': '20px', 'flex': '1'}, children=[
            html.H1('CS-499 Dashboard - Grazioso Salvare', style={'margin': '0', 'color': '#333'}),
            html.B("Database results from AAC Shelter | Developed by Andrew Allen")
        ])
    ]),
    
    html.Hr(style={'borderTop': '5px solid red', 'marginBottom': '20px'}),

    # FILTER SECTION: Interactive Radio Buttons
    html.Div(style={'marginBottom': '20px'}, children=[
        html.B("Select Rescue Filter Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Reset', 'value': 'button1'},
                {'label': 'Water Rescue', 'value': 'button2'},
                {'label': 'Mountain/Wilderness', 'value': 'button3'},
                {'label': 'Disaster Tracking', 'value': 'button4'}
            ],
            #default value is reset/button1 
            value='button1',
            labelStyle={'display': 'inline-block', 'marginRight': '15px', 'marginTop': '5px'}
        ),
    ]),

    # DATA TABLE SECTION HTML
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable="single",
        selected_rows=[0],
        filter_action="native",
        sort_action="native",
        style_header={'backgroundColor': 'white', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
        style_data_conditional=[{
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }]
    ),

    # VISUALIZATION ROW:
    #two different interactive visualazations. The left is a pie chart, and the right is a map which is explained later
    html.Div(style={'display': 'flex', 'flexDirection': 'row', 'marginTop': '20px', 'height': '500px'}, children=[
        
        # Left Side: Plotly Pie Chart
        html.Div(id='graph-container', style={
            'flex': '1', 
            'backgroundColor': 'white', 
            'borderRadius': '10px',
            'marginRight': '10px',
            'padding': '10px',
            'boxShadow': '2px 2px 5px rgba(0,0,0,0.1)'
        }, children=[
            html.Div(id='graph-id', style={'height': '100%'})
        ]),
        
        # Right Side: Interactive Leaflet Map
        html.Div(style={'flex': '1', 'backgroundColor': 'white', 'borderRadius': '10px', 'boxShadow': '2px 2px 5px rgba(0,0,0,0.1)'}, children=[
            dl.Map(id='map-id', center=[30.75, -97.48], zoom=15, children=[
                dl.TileLayer(),
                dl.LayerGroup(id='layer-group') 
            ], style={'width': '100%', 'height': '100%', 'borderRadius': '10px', 'cursor': 'crosshair'})
        ])
    ])
])

#############################################
# Controller Logic
#############################################

# TABLE FILTER CALLBACK
#Purpose: Filters based on necessary requirements provided for class. 
#Applies specified queries through mongoDB based on selection
# Implements specific Regex queries for Water, Mountain, and Disaster rescues
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    query = {}
    
    # Water Rescue: Lab/Chesapeake/Newfoundland, Intact Female, 26-156 weeks
    if filter_type == 'button2':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, 
            "sex_upon_outcome": "Intact Female", 
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    
    # Mountain/Wilderness: German Shepherd/Alaskan/Old English/Husky/Rottweiler, Intact Male, 26-156 weeks
    elif filter_type == 'button3':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, 
            "sex_upon_outcome": "Intact Male", 
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    
    # Disaster Tracking: Doberman/German Shepherd/Golden/Bloodhound/Rottweiler, Intact Male, 20-300 weeks
    elif filter_type == 'button4':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, 
            "sex_upon_outcome": "Intact Male", 
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    
    data = db.read(query)
    dff = pd.DataFrame.from_records(data)
    
    if not dff.empty and '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
        
    return dff.to_dict('records')


# PIE CHART CALLBACK
# Purpose:
#Visualization of the types of breeds located in the AAC database
#only the top 10 breeds are displayed, as too many results in an eldritch entity rather than a pie chart
# the lesser distribution of breeds is classified as 'other'
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graph(viewData):
    #Will not render if no data exists
    if not viewData:
        return []

    dff = pd.DataFrame(viewData)
    #Used to make sure the appropriate column "breed" exists, will not render otherwise
    if dff.empty or 'breed' not in dff.columns:
        return []
    # used to display top breeds on chart
    TOP_BREEDS = 10
    
    #counts the breeds, takes missing values and standardizes them
    rest_of_breeds = dff['breed'].fillna('Unknown').replace('', 'Unknown').value_counts()
    
    #used for displaying the breeds in the chart via displayed vs aggregated
    displayed = rest_of_breeds.head(TOP_BREEDS)
    not_displayed = rest_of_breeds.iloc[TOP_BREEDS:].sum()

    #if it can't be displayed, then it won't 
    pie_data = displayed.copy()
    if not_displayed > 0:
        pie_data.loc['Other'] = not_displayed

    pie_df = pie_data.reset_index()
    pie_df.columns = ['breed', 'count']

    #creates the pie chart
    fig = px.pie(
        pie_df,
        names='breed',
        values='count',
        title=f'Breed Distribution of Rescue Candidates (Top {TOP_BREEDS} + Other)'
    )

    # formatting
    fig.update_traces(textinfo='percent+label', textposition='inside')
    fig.update_layout(margin=dict(t=30, b=10, l=10, r=10))

    return dcc.Graph(figure=fig, style={'height': '100%'}, responsive=True)

# ADVANCED GEO-SPATIAL CALLBACK (K-D TREE INTEGRATION)
#This is the portion that relies on "data structures and algorithms"
# purpose:
#takes existing database data and vizualises it through the root node and its neighbors
# Synchronizes Map clicks with table row selections
#uses the data structure K-D tree to locate the nearest five neighbors in O(log n) time

@app.callback(
    [Output('map-id', 'center'),
     Output('layer-group', 'children'),
     Output('datatable-id', 'selected_rows')],
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows'),
     Input('map-id', 'click_lat_lng')]
)
def update_map(viewData, derived_selected_rows, click_lat_lng):
    #creates a center in texas, based on given coordinates
    default_center = [30.75, -97.48]
    #Guard clause:  if no data is given, returns Austin default
    if not viewData:
        return default_center, [], [0]
    #converts table's current data into dataframe for processing
    dff = pd.DataFrame(viewData)
    #if the dataframe is empty, the default position is given
    if dff.empty:
        return default_center, [], [0]
    
    required = {'location_lat', 'location_long'}
    if not required.issubset(set(dff.columns)):
        return default_center, [], [0]

    # Converts lat/longitude to numerics if given a string instead
    dff['location_lat'] = pd.to_numeric(dff['location_lat'], errors='coerce')
    dff['location_long'] = pd.to_numeric(dff['location_long'], errors='coerce')
    
    #if row cannot be used, it is dropped
    dff = dff.dropna(subset=['location_lat', 'location_long'])
    #if after conversions, no coordinates were legitmate, then default is returned
    if dff.empty:
        return default_center, [], [0]
    #CTX is callback contect, deciphers map click vs table click
    ctx = dash.callback_context
    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else None

    # IMPORTANT: reset index so DataTable selected_rows lines up with the dataframe position
    rows = dff.reset_index(drop=True)

    # Start from table selection
    target_pos = 0
    if derived_selected_rows:
        target_pos = max(0, min(int(derived_selected_rows[0]), len(rows) - 1))

    # Map click -> KDTree select nearest (to choose the selected row)
    if trigger_id == 'map-id' and click_lat_lng:
        if isinstance(click_lat_lng, (list, tuple)) and len(click_lat_lng) == 2:
            click_lat, click_lng = click_lat_lng
        elif isinstance(click_lat_lng, dict) and 'lat' in click_lat_lng and 'lng' in click_lat_lng:
            click_lat, click_lng = click_lat_lng['lat'], click_lat_lng['lng']
        else:
            click_lat, click_lng = None, None

        if click_lat is not None and click_lng is not None:
            coords = rows[['location_lat', 'location_long']].to_numpy()
            tree = KDTree(coords)
            _, tree_index = tree.query([click_lat, click_lng])
            target_pos = max(0, min(int(tree_index), len(rows) - 1))

    # -----------------------------
    # SHOW ONLY K NEAREST MARKERS
    # -----------------------------
    K = 6 # 6 is chosen to show the original + the 5 nearest neighbors
    coords_all = rows[['location_lat', 'location_long']].to_numpy()
    tree_all = KDTree(coords_all)
    #query K's nearest neighbors to the anchor
    anchor = coords_all[target_pos]
    k = min(K, len(rows))
    _, idxs = tree_all.query(anchor, k=k)

    #returns scalar if k==1
    if np.isscalar(idxs):
        idxs = [int(idxs)]
    else:
        idxs = [int(i) for i in idxs]

    markers = []
    #iloc is integer location indexing, which is necessary based on queries and data manipulation done
    # to the data frame
    for i in idxs:
        row = rows.iloc[i]
        lat, lon = float(row['location_lat']), float(row['location_long'])
        #for loop that iterates through the table and finds the chosen row, with i being the targeted position
        name_val = row.get('name', None)
        animal_name = name_val if pd.notna(name_val) and str(name_val).strip() != "" else "Unnamed"
        #populates the popup with information within the datatable from Mongo
        popup = dl.Popup([
            html.H4(f"Name: {animal_name}"),
            html.P(f"Animal ID: {row.get('animal_id', 'Unknown')}"),
            html.P(f"Breed: {row.get('breed', 'Unknown')}"),
            html.P(f"Age (Weeks): {row.get('age_upon_outcome_in_weeks', 'Unknown')}")
        ])
        tooltip = dl.Tooltip(animal_name)
        #highlights the targeted animal by using a ring/circle instead of a popup marker for neighbors
        if i == target_pos:
            markers.append(dl.CircleMarker(center=[lat, lon], radius=10, children=[tooltip, popup]))
        #standard popup otherwise
        else:
            markers.append(dl.Marker(position=[lat, lon], children=[tooltip, popup]))
    #where to put the center based on positioning taken from the dataframe
    center = [float(rows.iloc[target_pos]['location_lat']), float(rows.iloc[target_pos]['location_long'])]

    return center, markers, [target_pos]
if __name__ == '__main__':
    # Run in external mode for Jupyter compatibility
    app.run(jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/
